### An app to help drivers find a spot to park at UT

In [16]:
import sqlite3
import random
from datetime import datetime, timedelta

In [64]:
#db=sqlite3.connect("/Users/abhil/repo/park-ut/db/UTPark.db")
db=sqlite3.connect("./db/UTPark.db")

cursor = db.cursor()

In [99]:
def string_to_date(string):
    date = datetime.fromisoformat(string)
    print('# string_to_date - {} converted to {}'.format(string, date))
    return date

def row_to_user (row):
    user = {}
    user['id'] = row[0]
    user['name'] = row[1]
    user['email'] = row[2]
    user['admin'] = bool(row[3])
    print('# row_to_user - {} converted to {}'.format(row, user))
    return user

def row_to_reservation(row):
    res = {}
    res['id'] = row[0]
    res['spotid'] = row[1]
    res['expiry'] = string_to_date(row[2])
    return res

def fetch_all():
    all = cursor.fetchall()
    for pl in all:
        return(pl)

def fetch_users():
    admin = None
    non_admin = None
    cursor.execute('''SELECT * FROM users''')
    all = cursor.fetchall()
    for pl in all:
        print(pl)
        if non_admin is None and pl[3] is 0:
            non_admin = row_to_user(pl)
        elif admin is None and pl[3] is 1:
            admin = row_to_user(pl)
            
    return (non_admin, admin)
            
(non_admin, admin) = fetch_users()

print('non_admin: ', non_admin)
print('admin: ', admin)


(1, 'Vrinda', 'vrinda.sha@utexas.edu', 1)
# row_to_user - (1, 'Vrinda', 'vrinda.sha@utexas.edu', 1) converted to {'id': 1, 'name': 'Vrinda', 'email': 'vrinda.sha@utexas.edu', 'admin': True}
(2, 'Catherine', 'cathy.morgan@utexas.edu', 0)
# row_to_user - (2, 'Catherine', 'cathy.morgan@utexas.edu', 0) converted to {'id': 2, 'name': 'Catherine', 'email': 'cathy.morgan@utexas.edu', 'admin': False}
(3, 'Mac', 'macmiller@gmail.com', 0)
(4, 'Katie', 'Katie@gmail.com', 0)
(5, 'Abhi', 'Abhi@gmail.com', 0)
non_admin:  {'id': 2, 'name': 'Catherine', 'email': 'cathy.morgan@utexas.edu', 'admin': False}
admin:  {'id': 1, 'name': 'Vrinda', 'email': 'vrinda.sha@utexas.edu', 'admin': True}


In [19]:
def add_user(user, name, email, is_admin):
    # add new user
    if not (user['admin']):
        print('Insufficient access.')
        return
    
    try:
        insertsql = '''INSERT INTO users (Name,Email,Admin) VALUES ('{}', '{}', {})'''
        cursor.execute(insertsql.format(name, email, 1 if is_admin is True else False))
        db.commit()
        print('Successfully added a user.')
    except Exception as e:
        print(e)

add_user(non_admin, 'Abhi', 'Abhi@gmail.com', False)
add_user(admin, 'Abhi', 'Abhi@gmail.com', False)

Insufficient access.
UNIQUE constraint failed: users.Email


In [20]:
def add_garages(user, name, address):
    if not (user['admin']):
        print('Insufficient access.')
        return
    
    try:
        insertsql = '''INSERT INTO garages (Name, Address) VALUES ('{}', '{}')'''
        cursor.execute(insertsql.format(name, address))
        db.commit()
        print('Successfully added a garage.')
    except Exception as e:
        print(e)

add_garages(non_admin, 'Guadulupe', 'Guadulupe street')
add_garages(admin, 'Guadulupe', 'Guadulupe street')

Insufficient access.
Successfully added a garage.


In [21]:
def add_spots(user, garage_id, level):
    if not (user['admin']):
        print('Insufficient access.')
        return
    
    try:
        insertsql = '''INSERT INTO spots (GarageId, Level, Number) VALUES ({}, {}, {})'''
        cursor.execute(insertsql.format(garage_id, level, random.randint(1,10000)))
        db.commit()
        print('Successfully added a spot.')
    except Exception as e:
        print(e)

add_spots(non_admin, 1, 1)
add_spots(admin, 1, 1)

Insufficient access.
Successfully added a spot.


In [107]:
def get_reservation(spotid):
    cursor.execute('''SELECT * FROM reservations WHERE SpotId={}'''.format(spotid))
    reservation = cursor.fetchone()
    if reservation is None:
        print('# get_reservation - No reservation found.')
        return None
    
    reservation = row_to_reservation(reservation)
    now = datetime.now()
    
    if reservation['expiry'] < now:
        cursor.execute('''DELETE FROM reservations WHERE SpotId={}'''.format(spotid))
        print('# get_reservation - Expired reservation found. Deleted.')
        db.commit()
        return None
    else:
        print('# get_reservation - Returning {}'.format(reservation))
        return reservation
    
get_reservation(1)
get_reservation(2)

# get_reservation - No reservation found.
# get_reservation - No reservation found.


In [111]:
def make_reservation(user, spotid, onbehalfof=None):
    # this reserves a spot for parking for 10 minutes
    if onbehalfof is not None:
        if not (user['admin']):
            print('Insufficient access to reserve on behalf of another user.')
            return
        user = onbehalfof
    
    now = datetime.now()
    
    try:
        cursor.execute('''Select count(*) from occupancies where SpotId={}'''.format(spotid))
        is_Occ = cursor.fetchone()
        if is_Occ[0]!=0:
            print('Oops! This spot is taken! Try another!')
            return
            
        existing = get_reservation(spotid)
        if existing is not None:
            return

        expire = (datetime.now() + timedelta(minutes=10)).isoformat(timespec='minutes')
        insertsql = '''INSERT INTO reservations (SpotId, Expiry) VALUES ({}, '{}')'''
        cursor.execute(insertsql.format(spotid, expire))
        
        print('Successfully reserved till ', expire)
        
        db.commit()
    except Exception as e:
        db.rollback()
        print(e)
    
startevent(non_admin, 2)
startevent(non_admin, 2, non_admin)
startevent(admin, 2, non_admin)

Successfully reserved till -  2019-07-27T12:57
Insufficient access to reserve on behalf of another user.
Reservation already exists.


In [112]:
def occupancies(spotid):
    
    occStartTime = datetime.now()
    
    try:
        #first, check if spot is occupied
        cursor.execute('''Select * from occupancies where SpotId={}'''.format(spotid))
        existingoccupancy = cursor.fetchone()
        if existingoccupancy is not None:
            print('Spot is already occupied.')
            return
        
        #second, find the reservation
        reservation = get_reservation(spotid)
        if reservation is None:
            return
        
        #third, make occupancy
        insertsql = '''INSERT INTO occupancies (SpotId, StartTime) VALUES ({}, '{}')'''
        cursor.execute(insertsql.format(spotid, occStartTime))
        print('Awesome! We are glad to have you here!')

        #fourth, delete reservation
        deleteres = '''DELETE FROM reservations where SpotId={}'''.format(spotid)
        cursor.execute(deleteres)
        
        db.commit()
    except Exception as e:
        db.rollback()
        print(e)

occupancies(2)

# string_to_date - 2019-07-27T12:57 converted to 2019-07-27 12:57:00
# get_reservation - Returning {'id': 55, 'spotid': 2, 'expiry': datetime.datetime(2019, 7, 27, 12, 57)}
Awesome! We are glad to have you here!


In [109]:
def end_occupancy(spotid):
    cursor.execute('''SELECT * FROM occupancies where SpotId={}'''.format(spotid))
    occupancy = cursor.fetchone()
    if occupancy is None:
        print('Spot not occupied.')
    else:
        cursor.execute('''DELETE FROM occupancies where SpotId={}'''.format(spotid))
        print('Spot cleared.')
        db.commit()

end_occupancy(2)

Spot cleared.


In [79]:
cursor.execute("""Select count(*) from garages""")
num_garage = fetch_all()
print('Total number of garages - ',garage[0])

Total number of garages -  4


In [105]:
def totalSpots():
    cursor.execute("""Select count(*) from spots""")
    num_spot = fetch_all()
    print('Total number of spots - ',num_spot[0])
totalSpots()

Total number of spots -  7


In [107]:
def availableSpots():
    cursor.execute('''SELECT COUNT(*) FROM occupancies''')
    occ_Spots = fetch_all()
    avail_Spots = num_spot[0]-occ_Spots[0]
    print('Number of spots available - ',avail_Spots)

availableSpots()

Number of spots available -  -4


In [63]:
db.close()